In [1]:
import numpy as np
import pandas as pd
from tqdm import trange 
from tqdm import tqdm
import time
import os

In [2]:
df = pd.read_csv('study_room_HMS.csv')
df.head()

,Unnamed: 0,study_start_day,study_start_time,study_end_day,study_end_time,hashed
0,0,1,09:14:58.558,1,09:41:30.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...
1,1,17,11:05:05.176,17,13:07:42.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...
2,2,20,02:18:43.172,20,02:28:58.177,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...
3,3,22,09:22:01.936,22,09:47:40.192,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...
4,4,26,06:29:21.182,26,06:50:55.004,c87c2fad141edf323f3787335b54be22945a02fe052448...


In [40]:
## 각 사람 별로 index 지정

# 사람 별 index 변수
idx = dict()
# 사람 별 index
cnt = 0
for hsh in tqdm(df['hashed']):
    # 각 사람에 대해
    for user in hsh.split(","):
        # 사람이 idx에 존재할 경우 continue
        if user in idx:
            continue
        # user에 해당하는 index 부여
        idx[user] = cnt
        cnt += 1

100%|█████████████████████████████████████████████████████████████████████| 6962341/6962341 [02:05<00:00, 55586.87it/s]


In [41]:
len(idx)

268148

In [5]:
## 각 사람에 대한 총 이용 횟수 구하기

# 총 이용 횟수 변수
user_freq = [0] * len(idx)

for hsh in tqdm(df['hashed']):
    for user in hsh.split(","):
        # user의 index에 이용 횟수 더하기
        user_freq[idx[user]] += 1

100%|█████████████████████████████████████████████████████████████████████| 6962341/6962341 [01:45<00:00, 65749.40it/s]


In [ ]:
## 각 사람에 대한 총 이용 시간 구하기

"""
import datetime

for i in tqdm(range(len(df))):
    # time 변수를 str에서 datetime 객체로 만들기
    df['study_start_time'][i] = datetime.datetime.strptime(df['study_start_time'][i], '%H:%M:%S.%f')
    df['study_end_time'][i] = datetime.datetime.strptime(df['study_end_time'][i], '%H:%M:%S.%f')
    
    # 시간에 일수를 더해주기
    df['study_start_time'][i] += datetime.timedelta(days = int(df['study_start_day'][i]))
    df['study_end_time'][i] += datetime.timedelta(days = int(df['study_end_day'][i]))

"""

In [7]:
## 각 사람에 대한 총 이용 일 수

# 총 이용 일수 변수
user_day = [0] * len(idx)

for index, hsh in tqdm(enumerate(df['hashed'])):
    for user in hsh.split(","):
        user_day[idx[user]] += (df['study_end_day'][index] - df['study_start_day'][index] + 1)

6962341it [2:54:32, 664.82it/s]


In [9]:
## 각 사람에 대한 평균 같이 이용한 인원 수

# 평균 같이 이용한 인원 수 변수
user_together = [0] * len(idx)

for index, hsh in tqdm(enumerate(df['hashed'])):
    for user in hsh.split(","):
        # 자기 자신을 제외한 같이 이용한 인원 수 더하기
        user_together[idx[user]] += len(df['hashed'][index].split(",")) - 1

6962341it [1:26:18, 1344.52it/s]


In [10]:
for hsh in tqdm(df['hashed']):
    for user in hsh.split(","):
        # 총 같이 이용한 인원 수를 이용 횟수로 나누어 평균 같이 이용한 인원 수 구함
        user_together[idx[user]] /= user_freq[idx[user]]

100%|█████████████████████████████████████████████████████████████████████| 6962341/6962341 [03:05<00:00, 37552.13it/s]


In [8]:
## 각 사람의 요일별 이용일 수 (집계 시작일: 월요일, 시작일 기준)

# 요일 이용일수 변수
mon = [0] * len(idx)
tue = [0] * len(idx)
wed = [0] * len(idx)
thu = [0] * len(idx)
fri = [0] * len(idx)
sat = [0] * len(idx)
sun = [0] * len(idx)

for index, hsh in tqdm(enumerate(df['hashed'])):
    for user in hsh.split(","):
        # 각 요일에 해당하는 경우의 index 값에 횟수 더하기
        if df['study_start_day'][index] % 7 == 1:
            mon[idx[user]] += 1
        elif df['study_start_day'][index] % 7 == 2:
            tue[idx[user]] += 1
        elif df['study_start_day'][index] % 7 == 3:
            wed[idx[user]] += 1
        elif df['study_start_day'][index] % 7 == 4:
            thu[idx[user]] += 1
        elif df['study_start_day'][index] % 7 == 5:
            fri[idx[user]] += 1
        elif df['study_start_day'][index] % 7 == 6:
            sat[idx[user]] += 1
        else:
            sun[idx[user]] += 1

6962341it [6:49:55, 283.08it/s]


In [48]:
idx = pd.Series(list(idx.keys()))

In [12]:
user_freq = pd.Series(user_freq)

In [13]:
user_day = pd.Series(user_day)

In [14]:
user_together = pd.Series(user_together)

In [15]:
mon = pd.Series(mon)
tue = pd.Series(tue)
wed = pd.Series(wed)
thu = pd.Series(thu)
fri = pd.Series(fri)
sat = pd.Series(sat)
sun = pd.Series(sun)

In [49]:
data = pd.concat([idx, user_freq, user_day, user_together, mon, tue, wed, thu, fri, sat, sun], axis = 1,
                keys = ["idx", "user_freq", "user_day", "user_together", 
                           "mon", "tue", "wed", "thu", "fri", "sat", "sun"])

In [50]:
data.head()

,idx,user_freq,user_day,user_together,mon,tue,wed,thu,fri,sat,sun
0,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,129,135,2.595750e-270,9,15,16,21,23,23,22
1,a0b6ecbec654b18fe36ebe6230e25a653fb12125733583...,110,112,1.096707e-222,13,21,13,15,17,10,21
2,3193ab18168bcadbcb8342c06c4a35fa0d6e58d9619fe8...,479,487,0.000000e+00,35,52,60,86,84,89,73
3,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,267,276,0.000000e+00,19,34,35,36,50,46,47
4,769f00750d87fa59f8b6083aae62fb394e165101796fe5...,290,306,0.000000e+00,26,38,44,50,39,52,41
